In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

In [ ]:
df_bohdan = pd.read_csv('final_dataset.csv')
df_sofiia = pd.read_csv('rdi1_reduced/rdi.csv')
df_names = pd.read_csv('archive-diploma-kaggle/all_dist_villages.csv')

In [ ]:
df_names = df_names[["admin4Name", "admin4Na_1", "admin4Pcod"]]

In [ ]:
df_orig = df_bohdan.merge(df_sofiia, on='admin4Pcod', how='inner', suffixes=('_1', ''))
df_orig = df_orig.merge(df_names, on='admin4Pcod', how='inner', suffixes=('_1', ''))

In [ ]:
df = df_orig[['X', 'Y', 'quality', 'admin4Name', "admin4Pcod"]]
df = df.rename(columns={"admin4Pcod": "Postal code", "admin4Name": "Locality", 'quality': "RDL"})

df['RDL'] = (100 - df['RDL'] * 100).astype(int)

bins = [0, 30, 50, 70, 85, 100]
labels = ['Very Low', 'Low', 'Moderate', 'High', 'Very High']
df['RDL category'] = pd.cut(df['RDL'], bins=bins, labels=labels, include_lowest=True)

custom_data = df[['Locality', "Postal code", 'RDL']]

In [ ]:
colors = { 'Very Low': '#d7191c', 'Low': '#f5a153', 'Moderate': '#f7ff0a', 'High': '#8acc62', 'Very High': '#1a9641'}

fig = px.scatter(df, x='X', y='Y', color='RDL category', color_discrete_map=colors, hover_data={
        'X': False,  # Hide X
        'Y': False,  # Hide Y
        'RDL category': True,  # Hide RDI category
        'Locality': True,  # Show RDI (continuous data)
        'Postal code': True,  # Show Category
        'RDL': True  # Show Description
    })

hovertemplate = "<br>".join([
    "<b>%{customdata[1]}</b>",
    "%{customdata[2]}",
    "RDL: %{customdata[3]} (%{customdata[0]})<extra></extra>",
])

fig.update_traces(
    marker=dict(size=4),
    hovertemplate=hovertemplate,
    # showlegend=False,
)

fig.update_layout(
    # title='Sample scatter plot',
    plot_bgcolor="white",
    xaxis_title=None,
    xaxis_showticklabels=False,
    yaxis_title=None,
    yaxis_showticklabels=False
)

fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)
fig['layout'].update(margin=dict(l=0,r=0,b=0,t=24))


fig.show()

fig.write_html('rdl_graph_interactive.html')